In [5]:
import pandas as pd




In [6]:
# 등치지역도 (Choropletch)


In [7]:
# 법정동 코드 데이터
local_code = pd.read_excel(r'/Users/gamgoon/Development/ML/realty/KIKcd_B.20190701.xlsx')
# 법정동코드가 10자리로 되어있다. 읍면동의 법정코드까지 다 포함했을떄의 코드다. 시군구만 있으면 되기 때문에 앞의 5자리만
local_code['법정동코드'] = local_code['법정동코드'].apply(lambda x: str(x)[:5])
filtered_code = local_code[['법정동코드', '시도명', '시군구명']].drop_duplicates()
filtered_code.dropna(inplace=True)
filtered_code.loc[20477] = ('36110', '세종', '세종')

In [8]:
filtered_code

,법정동코드,시도명,시군구명
1,11110,서울특별시,종로구
89,11140,서울특별시,중구
164,11170,서울특별시,용산구
201,11200,서울특별시,성동구
219,11215,서울특별시,광진구
227,11230,서울특별시,동대문구
238,11260,서울특별시,중랑구
245,11290,서울특별시,성북구
285,11305,서울특별시,강북구
290,11320,서울특별시,도봉구


In [9]:
def get_sido(x):
    temp = x.split(' ')[0]
    if len(temp) != 4:
        return temp[:2]
    else:
        return temp[0] + temp[2]
    
filtered_code['시도명'] = filtered_code['시도명'].dropna().apply(get_sido)
filtered_code['시군구명'] = filtered_code['시군구명'].dropna().apply(lambda x: x.split(' ')[-1])

In [10]:
import xlwings as xw

def KBpriceindex_preprocessing(path, data_type):
    wb = xw.Book(path)
    sheet = wb.sheets[data_type]
    row_num = sheet.range('A1','A1').end('down').end('down').end('down').row
    data_range = 'A2:GE' + str(row_num)
    raw_data = sheet[data_range].options(pd.DataFrame, index=False, header=True).value
    bignames = '서울 대구 부산 대전 광주 인천 울산 세종 경기 강원 충북 충남 전북 전남 경북 경남 제주도 6개광역시 5개광역시 수도권 기타지방 구분 전국'
    bigname_list = bignames.split(' ')
    big_col = list(raw_data.columns)
    small_col = list(raw_data.iloc[0])
    for index, gu_data in enumerate(small_col):
        if gu_data == None:
            small_col[index] = big_col[index]
        check = index
        while True:
            if big_col[check] in bigname_list:
                big_col[index] = big_col[check]
                break
            else:
                check = check - 1
    big_col[129] = '경기'
    big_col[130] = '경기'
    small_col[185] = '서귀포'
    raw_data.columns = [big_col, small_col]
    new_col_data = raw_data.drop([0,1])
    new_col_data
    index_list = list(new_col_data['구분']['구분'])
    new_index = []
    for index, raw_index in enumerate(index_list):
        temp = str(raw_index).split('.')
        last = ''
        if int(temp[0]) > 12:
            if len(temp[0]) == 2:
                last = '19' + temp[0] + '.' + temp[1]
                new_index.append(last)
            else:
                last = temp[0] + '.' + temp[1]
                new_index.append(last)
        else:
            last = new_index[index - 1].split('.')[0] + '.' + temp[0]
            new_index.append(last)
    new_col_data.set_index(pd.to_datetime(new_index), inplace=True)
    cleaned_data = new_col_data.drop(('구분', '구분'), axis=1)
    return cleaned_data

In [11]:
path = r'/Users/gamgoon/Development/ML/realty/★(월간)KB주택가격동향_시계열(2019.05)21667280850300244.xls'
price_index = KBpriceindex_preprocessing(path, '매매apt')

In [12]:
# 특정 날짜를 기준으로 부동산 가격지수의 증감률 구하기
diff_pct = ((price_index.loc['2018-6-1'] / price_index.loc['2017-6-1']) - 1) * 100
diff_df = pd.DataFrame({'증감률': diff_pct})

In [13]:
diff_df

증감률
전국    전국      1.89857
서울    서울       9.1006
      강북      7.71459
      강북구     4.72529
      광진구     12.1218
      노원구     5.56922
      도봉구      7.0305
      동대문구    8.25455
      마포구     10.8554
      서대문구    8.08206
      성동구     13.4367
      성북구     6.99843
      용산구     10.8937
      은평구     4.76897
      종로구     5.98411
      중구      6.15858
      중랑구     5.57071
      강남       10.267
      강남구     14.1929
      강동구     11.9559
      강서구     6.36729
      관악구     6.88681
      구로구     5.80497
      금천구     5.81221
      동작구     10.7529
      서초구     8.10492
      송파구     14.0362
      양천구     9.20872
      영등포구    11.4094
6개광역시 6개광역시  0.617781
...               ...
전북    익산      1.67783
      군산     -3.54965
전남    전남      1.74565
      목포     0.852788
      순천      2.01498
      광양     0.925501
      여수      2.80215
경북    경북      -3.1634
      포항     -3.58169
      남구     -2.44961
      북구     -4.23524
      구미     -4.08329
      경산      -1.8078
      안동     -1.67041
      김천          NaN
경남    경남     -4.85277
      창원     -7.34855
      마산합포구  -6.67986
      마산회원구  -3.81608
      성산구    -10.4792
      의창구    -6.50921
      진해구    -6.50773
      양산     -1.41378
      거제      -7.4208
      진주     0.940335
      김해     -5.55731
      통영     -3.97497
제주도   제주도         NaN
      서귀포    -1.04049
기타지방  기타지방   -2.12976

[186 rows x 1 columns]

In [14]:
# 증감률 데이터프레임에 법정동 코드 데이터 추가
filtered_code.index = [filtered_code['시도명'], filtered_code['시군구명']]

code = []
for local in diff_df.index:
    if local[0] in filtered_code.index:
        temp_df = filtered_code.loc[local[0]]
        if local[1] in temp_df.index:
            code.append(temp_df.loc[local[1]]['법정동코드'])
        elif local[1] + '시' in temp_df.index:
            code.append(temp_df.loc[local[1] + '시']['법정동코드'])
        elif local[1] == '세종':
            code.append('36110')
        else:
            code.append('')
    else:
        code.append('')

diff_df['법정동코드'] = code

In [15]:
diff_df

증감률  법정동코드
전국    전국      1.89857       
서울    서울       9.1006       
      강북      7.71459       
      강북구     4.72529  11305
      광진구     12.1218  11215
      노원구     5.56922  11350
      도봉구      7.0305  11320
      동대문구    8.25455  11230
      마포구     10.8554  11440
      서대문구    8.08206  11410
      성동구     13.4367  11200
      성북구     6.99843  11290
      용산구     10.8937  11170
      은평구     4.76897  11380
      종로구     5.98411  11110
      중구      6.15858  11140
      중랑구     5.57071  11260
      강남       10.267       
      강남구     14.1929  11680
      강동구     11.9559  11740
      강서구     6.36729  11500
      관악구     6.88681  11620
      구로구     5.80497  11530
      금천구     5.81221  11545
      동작구     10.7529  11590
      서초구     8.10492  11650
      송파구     14.0362  11710
      양천구     9.20872  11470
      영등포구    11.4094  11560
6개광역시 6개광역시  0.617781       
...               ...    ...
전북    익산      1.67783  45140
      군산     -3.54965  45130
전남    전남      1.74565       
      목포     0.852788  46110
      순천      2.01498  46150
      광양     0.925501  46230
      여수      2.80215  46130
경북    경북      -3.1634       
      포항     -3.58169  47110
      남구     -2.44961  47111
      북구     -4.23524  47113
      구미     -4.08329  47190
      경산      -1.8078  47290
      안동     -1.67041  47170
      김천          NaN  47150
경남    경남     -4.85277       
      창원     -7.34855  48120
      마산합포구  -6.67986  48125
      마산회원구  -3.81608  48127
      성산구    -10.4792  48123
      의창구    -6.50921  48121
      진해구    -6.50773  48129
      양산     -1.41378  48330
      거제      -7.4208  48310
      진주     0.940335  48170
      김해     -5.55731  48250
      통영     -3.97497  48220
제주도   제주도         NaN       
      서귀포    -1.04049       
기타지방  기타지방   -2.12976       

[186 rows x 2 columns]

In [16]:
# 가격 증감률과 시도 경계선을 합쳐서 지도 위에 함께 나타내기
import json

rfile = open(r'/Users/gamgoon/Development/ML/realty/TL_SCCO_SIG_WGS84.json', 'r', encoding='utf-8').read()
jsonData = json.loads(rfile)

korea_map = folium.Map(location=[36, 127], zoom_start=7)

korea_map.choropleth(
    geo_data=jsonData,
    data=diff_df,
    columns=['법정동코드', '증감률'],
    key_on='feature.properties.SIG_CD',
    fill_color='RdYlGn',
    fill_opacity=0.7,
    line_opacity=0.5,
    legend_name='증감률(%'
)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/gamgoon/Development/ML/realty/TL_SCCO_SIG_WGS84.json'

In [ ]:
# HTML 파일로 저장
korea_map.save('부동산 증감 지도.html')
